In [148]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import warnings

warnings.simplefilter('ignore', ConvergenceWarning)
os.makedirs('graphs', exist_ok=True)

In [149]:
def load_prepare_data(file_path):
    # Load Domestic Currency sheet
    df = pd.read_excel(file_path, sheet_name='Domestic Currency', skiprows=5, header=[0,1])
    india_cols = [col for col in df.columns if col[1] == 'India']
    df_india = df[india_cols].copy()
    df_india['Items'] = df['Items']
    df_india = df_india.set_index('Items').T.reset_index()
    df_india = df_india.rename(columns={'level_0': 'Month', 'level_1': 'Country'})
    df_final = df_india[['Month', 'Country', 'Currency in Circulation']]
    df_final = df_final.iloc[16:].reset_index(drop=True)

    # Load Rates & Ratio sheet
    df_rates = pd.read_excel(file_path, sheet_name='Rates & ratio', skiprows=5, header=[0,1])
    india_cols_rate = [col for col in df_rates.columns if col[1] == 'India']
    df_india_rates = df_rates[india_cols_rate].copy()
    df_india_rates['Items'] = df_rates['Items']
    df_india_rates = df_india_rates.set_index('Items').T.reset_index()
    df_india_rates = df_india_rates.rename(columns={'level_0': 'Month', 'level_1': 'Country'})

    # Fix column names by stripping whitespace
    df_india_rates.columns = [col.strip() if isinstance(col, str) else col for col in df_india_rates.columns]

    needed_cols = ['Month', 'Country', 'Repo Rate', 'CPI Inflation Rate (in %)']
    missing_cols = [c for c in needed_cols if c not in df_india_rates.columns]
    if missing_cols:
        raise KeyError(f"Expected columns not found: {missing_cols}")

    df_final_rates = df_india_rates[needed_cols]
    df_final_rates = df_final_rates.iloc[16:].reset_index(drop=True)

    df_merged = pd.merge(df_final, df_final_rates, on=['Month', 'Country'], how='left')

    if len(df_merged) > 156:
        df_merged = df_merged.iloc[:-156]

    for col in ['Currency in Circulation', 'Repo Rate', 'CPI Inflation Rate (in %)']:
        df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

    df_merged.replace([np.inf, -np.inf], np.nan, inplace=True)

    df_merged[['Currency in Circulation', 'Repo Rate', 'CPI Inflation Rate (in %)']] = \
        df_merged[['Currency in Circulation', 'Repo Rate', 'CPI Inflation Rate (in %)']].interpolate(method='linear')

    df_merged.fillna(method='bfill', inplace=True)
    df_merged.fillna(method='ffill', inplace=True)

    df_merged['Month'] = pd.to_datetime(df_merged['Month'])
    df_merged = df_merged.sort_values('Month').reset_index(drop=True)

    if df_merged[['Repo Rate', 'CPI Inflation Rate (in %)']].isnull().any().any():
        print("Warning: NaNs remain in Repo Rate or CPI Inflation Rate after filling!")

    return df_merged

In [150]:
def save_summary_stats(df, filename='summary.txt'):
    with open(filename, 'w') as f:
        f.write('Summary Statistics\n')
        f.write('==================\n\n')
        f.write('Currency in Circulation:\n')
        f.write(str(df['Currency in Circulation'].describe()))
        f.write('\n\nRepo Rate:\n')
        f.write(str(df['Repo Rate'].describe()))
        f.write('\n\nCPI Inflation Rate (in %):\n')
        f.write(str(df['CPI Inflation Rate (in %)'].describe()))
        f.write('\n\nCorrelation Matrix:\n')
        corr = df[['Currency in Circulation', 'Repo Rate', 'CPI Inflation Rate (in %)']].corr()
        f.write(str(corr))
        f.write('\n\n')

In [151]:
def plot_time_series(df):
    plt.figure(figsize=(12,6))
    plt.plot(df['Month'], df['Currency in Circulation'], label='Currency in Circulation', color='tab:blue')
    plt.title('Currency in Circulation Over Time')
    plt.xlabel('Month')
    plt.ylabel('Currency in Circulation')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('graphs/currency_in_circulation.png')
    plt.close()

    plt.figure(figsize=(12,6))
    plt.plot(df['Month'], df['Repo Rate'], label='Repo Rate', color='tab:orange')
    plt.plot(df['Month'], df['CPI Inflation Rate (in %)'], label='CPI Inflation Rate (%)', color='tab:green')
    plt.title('Repo Rate and CPI Inflation Rate Over Time')
    plt.xlabel('Month')
    plt.ylabel('Rate / Inflation (%)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('graphs/repo_inflation_rates.png')
    plt.close()

In [152]:
def adf_test(series, col_name):
    result = adfuller(series.dropna())
    print(f'ADF Test for {col_name}:')
    print(f'  Test Statistic: {result[0]:.4f}')
    print(f'  p-value: {result[1]:.4f}')
    print('  Critical Values:')
    for key, value in result[4].items():
        print(f'    {key}: {value:.4f}')
    print('---\n')
    return result

In [153]:
def plot_acf_pacf(series, lags=40, prefix='currency'):
    fig, ax = plt.subplots(2,1, figsize=(12,8))
    plot_acf(series.dropna(), lags=lags, ax=ax[0])
    plot_pacf(series.dropna(), lags=lags, ax=ax[1])
    ax[0].set_title(f'ACF Plot for {prefix}')
    ax[1].set_title(f'PACF Plot for {prefix}')
    plt.tight_layout()
    plt.savefig(f'graphs/{prefix}_acf_pacf.png')
    plt.close()

In [154]:
def plot_decomposition(series, period=12, prefix='currency'):
    decomposition = seasonal_decompose(series.dropna(), model='additive', period=period)
    fig = decomposition.plot()
    fig.set_size_inches(12,8)
    plt.tight_layout()
    plt.savefig(f'graphs/{prefix}_decomposition.png')
    plt.close()

In [155]:
def sarimax_forecast(series, order=(1,0,0), seasonal_order=(0,0,0,0), steps=12):
    model = SARIMAX(series.dropna(), order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
    results = model.fit(disp=False)
    print(results.summary())
    forecast = results.get_forecast(steps=steps)
    forecast_ci = forecast.conf_int()

    plt.figure(figsize=(12,6))
    plt.plot(series.index, series, label='Observed')
    plt.plot(forecast.predicted_mean.index, forecast.predicted_mean, label='Forecast', color='red')
    plt.fill_between(forecast_ci.index, forecast_ci.iloc[:,0], forecast_ci.iloc[:,1], color='pink', alpha=0.3)
    plt.title('SARIMAX Forecast')
    plt.xlabel('Date')
    plt.ylabel(series.name)
    plt.legend()
    plt.tight_layout()
    plt.savefig('graphs/sarimax_forecast.png')
    plt.close()

    return results, forecast

In [156]:
# Main block to run
file_path = 'Monthly_Series_Economic_Variable.xlsx'  # Replace with your actual file path
df = load_prepare_data(file_path)
print(df.head())

save_summary_stats(df, 'summary.txt')
plot_time_series(df)

# Stationarity tests
adf_test(df['Currency in Circulation'], 'Currency in Circulation')
adf_test(df['Repo Rate'], 'Repo Rate')
adf_test(df['CPI Inflation Rate (in %)'], 'CPI Inflation Rate')

# Plots
plot_acf_pacf(df['Currency in Circulation'], prefix='currency')
plot_decomposition(df['Currency in Circulation'], prefix='currency')

# SARIMAX Forecast (example) - order and seasonal_order should be chosen carefully after model identification
sarimax_forecast(df.set_index('Month')['Currency in Circulation'], order=(1,1,1), seasonal_order=(1,1,1,12), steps=12)

/usr/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/tmp/ipykernel_5624/1151667768.py:44: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged.fillna(method='bfill', inplace=True)
/tmp/ipykernel_5624/1151667768.py:45: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged.fillna(method='ffill', inplace=True)
/tmp/ipykernel_5624/1151667768.py:47: UserWarning: Could not infer format, so each element will be parsed individually, fallin

       Month Country  Currency in Circulation  Repo Rate  \
0 2014-01-01   India             1.279164e+07        8.0   
1 2014-02-01   India             1.291939e+07        8.0   
2 2014-03-01   India             1.301074e+07        8.0   
3 2014-04-01   India             1.345088e+07        8.0   
4 2014-05-01   India             1.365730e+07        8.0   

   CPI Inflation Rate (in %)  
0                   8.604207  
1                   7.882241  
2                   8.246445  
3                   8.482564  
4                   8.325538  
ADF Test for Currency in Circulation:
  Test Statistic: -0.3670
  p-value: 0.9155
  Critical Values:
    1%: -3.4875
    5%: -2.8866
    10%: -2.5801
---

ADF Test for Repo Rate:
  Test Statistic: -1.8710
  p-value: 0.3458
  Critical Values:
    1%: -3.4880
    5%: -2.8868
    10%: -2.5802
---

ADF Test for CPI Inflation Rate:
  Test Statistic: -1.4208
  p-value: 0.5723
  Critical Values:
    1%: -3.4930
    5%: -2.8890
    10%: -2.5814
---



/usr/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


                                     SARIMAX Results                                      
Dep. Variable:            Currency in Circulation   No. Observations:                  120
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 12)   Log Likelihood               -1400.706
Date:                            Mon, 28 Apr 2025   AIC                           2811.413
Time:                                    17:12:06   BIC                           2824.076
Sample:                                01-01-2014   HQIC                          2816.526
                                     - 12-01-2023                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.3032      0.434      0.699      0.484      -0.547       1.153
ma.L1          0.0739      0.445   

(<statsmodels.tsa.statespace.sarimax.SARIMAXResultsWrapper at 0x72b9ef708440>,
 <statsmodels.tsa.statespace.mlemodel.PredictionResultsWrapper at 0x72b9ef708980>)